In [1]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\t0263519\\Desktop\\Study\\PYTHON\\MLFLOW-Projects\\TextSummarizer'

In [8]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataTransformationConfig:
    root_dir: Path
    tokenizer_name: Path
    data_path: Path

In [9]:
from src.TextSummarizer.utils.common import read_yaml,create_directories
from src.TextSummarizer.constants import *

In [10]:
class ConfigurationManager():
    def __init__(self, config_file=CONFIG_FILE_PATH, param_file=PARAM_FILE_PATH):
        self.config=read_yaml(config_file)
        self.param=read_yaml(param_file)
        create_directories([self.config.artifacts_root])

    def get_data_transformation(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            tokenizer_name=config.tokenizer_name,
            data_path=config.data_path
        )
        return data_transformation_config
    



In [11]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk


In [14]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsun_dataset"))

In [15]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation()
data_transformation=DataTransformation(config=data_transformation_config)
#data_transformation.convert_examples_to_features()
data_transformation.convert()

[2025-02-14 04:31:11,146: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-02-14 04:31:11,148: INFO: common: yaml file params.yaml loaded successfully]
[2025-02-14 04:31:11,149: INFO: common: artifacts directory created]
[2025-02-14 04:31:11,150: INFO: common: artifacts/data_transformation directory created]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\t0263519\Desktop\Study\PYTHON\MLFLOW-Projects\TextSummarizer\venv\lib\site-packages\transformers\tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 163604.06 examples/s]
